In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
%cd /content/drive/MyDrive/bootcamp/Averiguacao Cadastral

/content/drive/MyDrive/bootcamp/Averiguacao Cadastral


In [3]:
%ls

Análises/  base_pre/  bases/  catboost_info/  Dados/  Ensaios/  Limpeza/


## Carregando as Bases de dados

In [4]:
# imports

import pandas as pd
import numpy as np

## Preparação da base

In [5]:
# Carregando o dataset FAMILIAS/pessoas

pd.set_option('display.max_columns', 51)
pd.set_option('display.max_rows', 51)

df = pd.read_csv('bases/df_fam_pessoas_amostra_100mil.csv', sep=',')
print(f'Shape:', df.shape)

df.head()

Shape: (254997, 51)


,cd_ibge,uf_ibge,regiao_ibge,estrato,classf,id_familia,vlr_renda_media_fam,cod_local_domic_fam,cod_especie_domic_fam,qtd_comodos_domic_fam,qtd_comodos_dormitorio_fam,cod_material_piso_fam,cod_material_domic_fam,cod_agua_canalizada_fam,cod_abaste_agua_domic_fam,cod_banheiro_domic_fam,cod_escoa_sanitario_domic_fam,cod_destino_lixo_domic_fam,cod_iluminacao_domic_fam,cod_calcamento_domic_fam,cod_familia_indigena_fam,ind_familia_quilombola_fam,ind_parc_mds_fam,qtde_pessoas,peso.fam,dias_cadastramento,dias_atualizacao,classe_renda,id_pessoa,cod_sexo_pessoa,idade,cod_parentesco_rf_pessoa,cod_raca_cor_pessoa,cod_local_nascimento_pessoa,cod_certidao_registrada_pessoa,cod_deficiencia_memb,cod_sabe_ler_escrever_memb,ind_frequenta_escola_memb,cod_escola_local_memb,cod_curso_frequenta_memb,cod_ano_serie_frequenta_memb,cod_curso_frequentou_pessoa_memb,cod_ano_serie_frequentou_memb,cod_concluiu_frequentou_memb,cod_trabalhou_memb,cod_afastado_trab_memb,cod_agricultura_trab_memb,cod_principal_trab_memb,cod_trabalho_12_meses_memb,qtd_meses_12_meses_memb,peso.pes
0,4212908,42,4,1,2,268503,60.0,1.0,1.0,5.0,2.0,5.0,1.0,1.0,1.0,1.0,3.0,1.0,1.0,1.0,2.0,2.0,0.0,4,5502526707828,5128.0,102,0.0,2351897,2,4,3,1.0,2.0,1,2,2.0,1.0,1.0,2.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,563277716385694
1,4212908,42,4,1,2,268503,60.0,1.0,1.0,5.0,2.0,5.0,1.0,1.0,1.0,1.0,3.0,1.0,1.0,1.0,2.0,2.0,0.0,4,5502526707828,5128.0,102,0.0,2578989,2,15,3,1.0,1.0,1,2,1.0,1.0,1.0,7.0,1.0,-1.0,-1.0,-1.0,2.0,2.0,-1.0,-1.0,2.0,-1.0,563277716385694
2,4212908,42,4,1,2,268503,60.0,1.0,1.0,5.0,2.0,5.0,1.0,1.0,1.0,1.0,3.0,1.0,1.0,1.0,2.0,2.0,0.0,4,5502526707828,5128.0,102,0.0,3312847,2,13,3,1.0,1.0,1,2,2.0,1.0,1.0,5.0,7.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,563277716385694
3,4212908,42,4,1,2,268503,60.0,1.0,1.0,5.0,2.0,5.0,1.0,1.0,1.0,1.0,3.0,1.0,1.0,1.0,2.0,2.0,0.0,4,5502526707828,5128.0,102,0.0,10482075,2,38,1,1.0,2.0,1,2,1.0,3.0,-1.0,-1.0,-1.0,4.0,4.0,1.0,1.0,-1.0,1.0,6.0,1.0,12.0,563277716385694
4,2931905,29,2,2,3,2888440,0.0,2.0,1.0,2.0,1.0,2.0,2.0,2.0,4.0,1.0,3.0,3.0,1.0,3.0,2.0,2.0,0.0,2,550282046697327,2646.0,151,0.0,5553031,2,5,3,4.0,2.0,1,2,2.0,1.0,1.0,5.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,563263195326236


In [6]:
# DELETANDO COLUNAS DESNECESSÁRIAS
print(f'Shape antes:', df.shape)
df.drop('cd_ibge', axis=1, inplace=True)
df.drop('regiao_ibge', axis=1, inplace=True)
df.drop('vlr_renda_media_fam', axis=1, inplace=True)
df.drop('peso.fam', axis=1, inplace=True)
df.drop('peso.pes', axis=1, inplace=True)
print(f'Shape depois:', df.shape)

Shape antes: (254997, 51)
Shape depois: (254997, 46)


In [7]:
# CRIANDO LISTA DE VARIÁVEIS POR TEMA PARA FACILITAR MANIPULAÇÃO E TRATAMENTO

var_id = ['id_familia', 'id_pessoa', 'ind_parc_mds_fam']
var_pes = ['qtde_pessoas', 'cod_sexo_pessoa', 'cod_parentesco_rf_pessoa', 'idade',
       'cod_raca_cor_pessoa', 'cod_local_nascimento_pessoa',
       'cod_certidao_registrada_pessoa', 'cod_deficiencia_memb']

## Tratando características das pessoas

In [8]:
# filtrando base
selected_columns = list(set(var_id + var_pes))
df1 = df[selected_columns]
df1

,id_familia,cod_local_nascimento_pessoa,idade,cod_raca_cor_pessoa,cod_deficiencia_memb,id_pessoa,qtde_pessoas,cod_certidao_registrada_pessoa,cod_sexo_pessoa,cod_parentesco_rf_pessoa,ind_parc_mds_fam
0,268503,2.0,4,1.0,2,2351897,4,1,2,3,0.0
1,268503,1.0,15,1.0,2,2578989,4,1,2,3,0.0
2,268503,1.0,13,1.0,2,3312847,4,1,2,3,0.0
3,268503,2.0,38,1.0,2,10482075,4,1,2,1,0.0
4,2888440,2.0,5,4.0,2,5553031,2,1,2,3,0.0
...,...,...,...,...,...,...,...,...,...,...,...
254992,1330889,1.0,33,4.0,2,999566,3,1,1,2,0.0
254993,1330889,1.0,37,4.0,2,5315031,3,1,2,1,0.0
254994,1330889,2.0,6,4.0,2,12707309,3,1,1,3,0.0
254995,1481101,1.0,65,2.0,2,4367533,2,2,1,2,0.0


In [9]:
df1.columns

Index(['id_familia', 'cod_local_nascimento_pessoa', 'idade',
       'cod_raca_cor_pessoa', 'cod_deficiencia_memb', 'id_pessoa',
       'qtde_pessoas', 'cod_certidao_registrada_pessoa', 'cod_sexo_pessoa',
       'cod_parentesco_rf_pessoa', 'ind_parc_mds_fam'],
      dtype='object')

In [10]:
# CRIANDO DF VARIÁVEIS PESSOAS ANALISADAS POR FAMILIA

# Nova função para calcular a existência da característica
def verifica_caracteristica(x, condicao):
    return 1 if condicao.any() else 0

# Calcular percentual pessoas com determinada característica por familia
df_var_caracterisca_pessoas = df1.groupby('id_familia').apply(
    lambda x: pd.Series({
        'qde_pessoas': x['qtde_pessoas'].iloc[0],  # mesma quantidade por id_familia
        'pct_mulheres': (x['cod_sexo_pessoa'] == 2).sum() / x['qtde_pessoas'].iloc[0],
        'pct_homens': (x['cod_sexo_pessoa'] == 1).sum() / x['qtde_pessoas'].iloc[0],
        'pct_1_infancia': (x['idade'] <= 6).sum() / x['qtde_pessoas'].iloc[0],
        'pct_crianca_adolescente': ((x['idade'] > 6) & (x['idade'] <= 17)).sum() / x['qtde_pessoas'].iloc[0],
        'pct_adulto': ((x['idade'] > 18) & (x['idade'] < 60)).sum() / x['qtde_pessoas'].iloc[0],
        'pct_idosos': (x['idade'] >= 60).sum() / x['qtde_pessoas'].iloc[0],
        'pct_deficientes': (x['cod_deficiencia_memb'] == 1).sum() / x['qtde_pessoas'].iloc[0],
        # Acrescentado por Grinaldo

        'qtd_mulheres': (x['cod_sexo_pessoa'] == 2).sum(),
        'qtd_homens': (x['cod_sexo_pessoa'] == 1).sum(),
        'qtd_1_infancia': (x['idade'] <= 6).sum(),
        'qtd_crianca_adolescente': ((x['idade'] > 6) & (x['idade'] <= 17)).sum(),
        'qtd_adulto': ((x['idade'] > 18) & (x['idade'] <= 60)).sum(),
        'qtd_idosos': (x['idade'] >= 60).sum(),
        'qtd_deficientes': (x['cod_deficiencia_memb'] == 1).sum(),

        'tem_mulher': verifica_caracteristica(x, x['cod_sexo_pessoa'] == 2),
        'tem_homem': verifica_caracteristica(x, x['cod_sexo_pessoa'] == 1),
        'tem_1_infancia': verifica_caracteristica(x, x['idade'] <= 6),
        'tem_crianca_adolescente': verifica_caracteristica(x, (x['idade'] > 6) & (x['idade'] <= 17)),
        'tem_adulto': verifica_caracteristica(x, (x['idade'] > 18) & (x['idade'] <= 60)),
        'tem_idoso': verifica_caracteristica(x, x['idade'] >= 60),
        'tem_deficiente': verifica_caracteristica(x, x['cod_deficiencia_memb'] == 1)
    })
    # Removing include_groups as it's deprecated
).reset_index()

df_var_caracterisca_pessoas

,id_familia,qde_pessoas,pct_mulheres,pct_homens,pct_1_infancia,pct_crianca_adolescente,pct_adulto,pct_idosos,pct_deficientes,qtd_mulheres,qtd_homens,qtd_1_infancia,qtd_crianca_adolescente,qtd_adulto,qtd_idosos,qtd_deficientes,tem_mulher,tem_homem,tem_1_infancia,tem_crianca_adolescente,tem_adulto,tem_idoso,tem_deficiente
0,14,3.0,0.333333,0.666667,0.333333,0.000000,0.666667,0.0,0.0,1.0,2.0,1.0,0.0,2.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0
1,36,4.0,0.750000,0.250000,0.250000,0.250000,0.500000,0.0,0.0,3.0,1.0,1.0,1.0,2.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0
2,57,4.0,0.500000,0.500000,0.000000,0.250000,0.750000,0.0,0.0,2.0,2.0,0.0,1.0,3.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0
3,65,4.0,0.500000,0.500000,0.000000,0.250000,0.750000,0.0,0.0,2.0,2.0,0.0,1.0,3.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0
4,124,3.0,0.333333,0.666667,0.666667,0.000000,0.333333,0.0,0.0,1.0,2.0,2.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96497,5290445,3.0,1.000000,0.000000,0.000000,0.666667,0.333333,0.0,0.0,3.0,0.0,0.0,2.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0
96498,5290509,3.0,0.666667,0.333333,0.333333,0.333333,0.333333,0.0,0.0,2.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0
96499,5290542,4.0,0.250000,0.750000,0.250000,0.250000,0.500000,0.0,0.0,1.0,3.0,1.0,1.0,2.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0
96500,5290562,2.0,0.500000,0.500000,0.000000,0.000000,0.000000,1.0,0.0,1.0,1.0,0.0,0.0,0.0,2.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0


In [11]:
# Salvando base
df_var_caracterisca_pessoas.to_csv('base_pre/df_var_caracterisca_pessoas_grinaldo.csv', index=False)


Obs: A realizar o merge desta base com df_familia_pessoa_final.csv, deve ser executado previamente as seguintes linhas de código para alterar o valor de ind_parc_mds_fam:

import numpy as np

df_familia_pessoas_final['ind_parc_mds_fam'] = np.where(df_familia_pessoas_final['ind_parc_mds_fam'] != 0, 1, 0)